In [22]:
"""
Function to compute the execution time.
"""
from datetime import timedelta
import time
def get_time(f, kwargs={}):
    times = []
    result = None
    for i in range(5):
        print("measure",i+1)
        t = time.time()
        result = f(**kwargs)
        times.append(time.time()-t)
        
    print("all times:", times)
    a = min(times)
    b = max(times)
    print("times' outliers:",a,"&",b)
    t = (sum(times)-a-b)/3
    print("benchmark time",str(t)+"s","or", timedelta(seconds=t))
    return result

In [23]:
import skmob
import pandas as pd
import folium
from skmob.preprocessing import compression, filtering, detection
from geopy.distance import geodesic 
from shapely.geometry import Point, LineString, Polygon
import geopandas as gpd

In [24]:
DATASET_SIZE = "sample"

# Cars trajectories

In [25]:
PATH='../datasets/walking_journey (copy).csv'
df = pd.read_csv(PATH)
trajectory_df = skmob.TrajDataFrame(data=df, latitude='latitude', longitude='longitude', trajectory_id='id', datetime='time')

### Stop detection 

In [26]:
# Stop point detection
stops = detection.stay_locations(tdf=trajectory_df,minutes_for_a_stop=10,spatial_radius_km=0.040)
print("nombre de stops detectés:",len(stops))
stops

nombre de stops detectés: 3


,datetime,lat,lng,tid,leaving_datetime
0,2023-12-23 14:29:27,50.836402,4.363014,0,2023-12-23 14:42:31
1,2023-12-23 14:47:19,50.837425,4.361681,0,2023-12-23 14:57:43
2,2023-12-23 15:09:27,50.835927,4.362524,0,2023-12-23 15:31:35


In [27]:
HM = (50.83749695938153, 4.361910678357021)
ICI_PARIS = (50.83639483000942, 4.36310730085289)
PRIMARK = (50.83609678331181, 4.362912067372084)

hm = (stops["lat"].values[0], stops["lng"].values[0])
ici_paris = ((stops["lat"].values[1], stops["lng"].values[1]))
primark = ((stops["lat"].values[2], stops["lng"].values[2]))
print(hm, ici_paris, primark)

def dist(coord1, coord2):
    return geodesic(coord1, coord2).m

dist(hm, HM), dist(ici_paris, ICI_PARIS), dist(primark, PRIMARK)

(50.83640209682035, 4.363014126739705) (50.83742489870556, 4.3616806) (50.8359266, 4.362524126154138)


(144.48823186031183, 152.4188630060432, 33.24556496099119)

In [31]:
m=trajectory_df.plot_trajectory(tiles="OpenStreetMap", zoom=18.4)
m.add_child(folium.Marker(location=hm))
m.add_child(folium.Marker(location=ici_paris))
m.add_child(folium.Marker(location=primark))

hm_points = [(4.361751392226751,50.837557454216),
(4.361680197932412,50.83758512529476),
(4.361615507694445,50.83749602253124),
(4.361348909126123,50.83758455842273),
(4.361286136778264,50.83746780017795),
(4.361574839843958,50.83725473799701),
(4.36203388271873,50.8374328390111),
(4.361942032549679,50.8375707149803),
(4.361751392226751,50.837557454216)]

primark_points = [(4.362896766180572,50.83617392790824),
(4.362470536721625,50.83596462800733),
(4.362415641019282,50.83600564607535),
(4.361831949209036,50.83568922774191),
(4.36214474170608,50.83547792998715),
(4.362863512141311,50.83587392823398),
(4.3628074875803,50.83591228182476),
(4.363039276765206,50.83603882377207),
(4.362896766180572,50.83617392790824),
]

ici_paris_points = [(4.362977955590129,50.83630268153196),
(4.363328861404733,50.83639980218501),
(4.363276140239741,50.8364930460425),
(4.362918451023474,50.83639022943366),
(4.362977955590129,50.83630268153196)]

hm_polygon = Polygon(hm_points)
primark_polygon = Polygon(primark_points)
ici_paris_polygon = Polygon(ici_paris_points)
d = {'geometry': [hm_polygon,primark_polygon,ici_paris_polygon]}
gdf = gpd.GeoDataFrame(d, crs="EPSG:31370")

for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"])#.simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    #folium.Popup(r["BoroName"]).add_to(geo_j)
    geo_j.add_to(m)
print(sim_geo.to_json())
m

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[4.362977955590129, 50.83630268153196], [4.363328861404733, 50.83639980218501], [4.363276140239741, 50.8364930460425], [4.362918451023474, 50.83639022943366], [4.362977955590129, 50.83630268153196]]]}, "bbox": [4.362918451023474, 50.83630268153196, 4.363328861404733, 50.8364930460425]}], "bbox": [4.362918451023474, 50.83630268153196, 4.363328861404733, 50.8364930460425]}


/home/pierre-cedric/miniconda3/envs/skmob/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,
/home/pierre-cedric/miniconda3/envs/skmob/lib/python3.8/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: If necessary, trajectories will be down-sampled to have at most `max_points` points. To avoid this, specify `max_points=None`.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,


In [29]:
m.save("img.html")